# Building a Comedy Show Travel Planner with Hyperbrowser

Let's build a smart travel planner that combines AI planning with web automation to find comedy shows and flights. This agent will:

- Break down complex travel planning into simple steps
- Search for shows and check ticket availability
- Find matching flights based on show dates
- Keep track of progress across multiple sites

No more juggling between tabs or losing track of your planning process!

## Prerequisites

You'll need:
1. A Hyperbrowser API key (sign up at [hyperbrowser.ai](https://hyperbrowser.ai))
2. An Anthropic API key
3. Python 3.9+

Store your API keys in a `.env` file:
```shell
HYPERBROWSER_API_KEY=your_hyperbrowser_key_here
ANTHROPIC_API_KEY=your_anthropic_key_here
```

In [ ]:
import os
import json
from typing import List

from dotenv import load_dotenv
from hyperbrowser import AsyncHyperbrowser
from hyperbrowser.models import (
    StartBrowserUseTaskParams,
    CreateSessionParams,
    ScreenConfig,
)
from anthropic import AsyncAnthropic
from anthropic.types import MessageParam

from pydantic import BaseModel

from IPython.display import Markdown, display

load_dotenv()

## Initialize Clients

Set up both Hyperbrowser for web automation and Anthropic for task planning.

In [7]:
hb = AsyncHyperbrowser(api_key=os.getenv("HYPERBROWSER_API_KEY"))
llm = AsyncAnthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))

## Smart Task Planning

First, we'll break down our travel planning into manageable steps. This helps ensure we:
- Don't miss any important details
- Handle tasks in the right order
- Keep track of progress
- Can recover from any failures

Usually, Browser Use already has a planner process that it uses internally, but for more complex steps, a separate planner that lets us review steps before execution can be helpful. Smaller steps are easier to process and handle, so this makes sure we give our Browser Use instance the best shot at success!

In [8]:
class TaskSteps(BaseModel):
    step_info: str
    sub_steps: list[str]


class Task(BaseModel):
    task: str
    steps: list[TaskSteps]
    end_response: str


async def detailed_plan(user_content: str):
    messages: List[MessageParam] = []
    messages = [
        {
            "role": "assistant",
            "content": f"""You have to formulate a step by step plan for a given user task. Break it down into individual chunks, which order they'd have to be performed in, and within each task, what sub steps would have to be done. Keep the text instructions simple, enough so that a 5 year old kid could understand and perform them. Be detailed, but concise. If you have to search for something, don't be too specific, but rather add steps so that the information gets filtered there. The response should have the overall task information, the steps information, the sub-steps in each step and the end-result to be achieved.
                        
            Respond with only the task details and steps, no other information is required.

            The response format should be list this : {Task.model_json_schema()}

            Return only the json information itself. Don't put any ``` or ```json or anything similar anywhere.
            """,
        },
        {"role": "user", "content": user_content},
    ]

    process_plan = await llm.messages.create(
        model="claude-3-7-sonnet-latest",
        messages=messages,
        max_tokens=4000,
    )
    return process_plan

In [9]:
async def perform_task(
    overall_task: str,
    current_step_info: str,
    previous_steps_done: list[str],
    session_id: str,
):
    resp = await hb.agents.browser_use.start_and_wait(
        StartBrowserUseTaskParams(
            task=f"You are doing a multi-step process. Your final task is to \"{overall_task}\". Currently, you're doing a single step in that process - \"{current_step_info}\". Here are the steps that you've already done - \n{"\n-".join(previous_steps_done)}\n. If you fail to complete the step for whatever reason, just respond with \"Failure\", and nothing else.",
            use_vision=True,
            session_id=session_id,
            keep_browser_open=True,
            max_steps=10,
            max_actions_per_step=10
        )
    )
    if resp.data is not None:
        return resp.data.final_result
    return None

## Let's Plan a Trip!

Watch as our agent:
1. Creates a detailed plan
2. Executes each step systematically
3. Keeps you updated on progress
4. Finds the best combination of show tickets and flights

In [11]:
task = "Find me tickets for John mulaney sometime in June 2025. Also find but don't book me flights tickets to go to that place. I'm travelling from Cincinnati. Try to keep the ticket prices low. Look for flight tickets two days arriving two days before the event, and leaving one day after the event. "
plan = await detailed_plan(task)
if plan.content[0].type == "text":
    formatted_plan = plan.content[0].text
    print(formatted_plan)
    task = Task.model_validate(json.loads(formatted_plan))
    steps_done: list[str] = []

    session = await hb.sessions.create(
        CreateSessionParams(screen=ScreenConfig(width=1920, height=1080))
    )

    session_id = session.id

    print(session.live_url)
    try:
        for step in task.steps:
            response = await perform_task(
                task.task,
                f"{step.step_info}. The sub-steps for this step are - \n{"\n-".join(step.sub_steps)}",
                steps_done,
                session_id,
            )
            if response == "Failure":
                raise Exception(f"Could not complete step - {step.step_info}")
            steps_done.append(f"Step => {step.step_info}. Task Done => {response}")
            if response is not None:
                display(Markdown(response))
            else:
                print("No response from the agent")
    except Exception as e:
        print(e)
    finally:
        await hb.sessions.stop(session_id)
else:
    print(None)

{
    "task": "Find tickets for John Mulaney show in June 2025 and check flight options from Cincinnati",
    "steps": [
        {
            "step_info": "Search for John Mulaney shows in June 2025",
            "sub_steps": [
                "Open your computer or phone",
                "Go to a search engine like Google",
                "Type 'John Mulaney tour June 2025'",
                "Look at the results to find where he's performing",
                "Write down the cities and dates of his shows"
            ]
        },
        {
            "step_info": "Pick a John Mulaney show to attend",
            "sub_steps": [
                "Look at all the cities and dates from your list",
                "Choose a show date that works for you in June 2025",
                "Write down the exact date, time, and city of your chosen show"
            ]
        },
        {
            "step_info": "Find tickets for the John Mulaney show",
            "sub_steps": [
              

I found one show for John Mulaney in June 2025. It is in Newark, NJ on June 27, 2025. Now I will search for tickets for the show and flight options from Cincinnati.

I found one show for John Mulaney in June 2025. It is in Newark, NJ on June 27, 2025. Now I will search for tickets for the show and flight options from Cincinnati.

I found tickets for the John Mulaney show on June 27, 2025, in Newark, NJ. Cheapest tickets are $72.50 in Sec 107, Row 13 and Row 14. I did not check flight options from Cincinnati because I ran out of steps.

The show is on June 27, 2025. Arrival date is June 25, 2025. Departure date is June 28, 2025. Now I will search for flight options from Cincinnati.

I have found tickets for the John Mulaney show on June 27, 2025, in Newark, NJ. Cheapest tickets are $72.50 in Sec 107, Row 13 and Row 14. The show is in Newark, NJ on June 27, 2025. Arrival date is June 25, 2025. Departure date is June 28, 2025. I have also searched for flight options from Cincinnati to Newark, NJ on Google Flights. Flights from Cincinnati to Newark from June 25, 2025 to June 28, 2025 were found.

I have found tickets for the John Mulaney show on June 27, 2025, in Newark, NJ. Cheapest tickets are $72.50 in Sec 107, Row 13 and Row 14. The show is in Newark, NJ on June 27, 2025. Arrival date is June 25, 2025. Departure date is June 28, 2025. I have also searched for flight options from Cincinnati to Newark, NJ on Google Flights. The cheapest flight option is $177 round trip with Delta, leaving Cincinnati at 7:05 AM on June 25 and arriving in Newark at 9:00 AM. Another option is at 12:30 PM, arriving at 2:28 PM, also for $177. A third option is at 5:42 PM, arriving at 7:41 PM, also for $177.

## Make It Your Own

This pattern works for any event planning:
- Different performers or shows
- Various departure cities
- Custom date ranges
- Specific planning requirements

## Conclusion

We've built a powerful travel planning agent that combines AI planning with web automation to:
- Break down complex travel searches into simple steps
- Search multiple sites for show tickets and flights
- Track progress and maintain context between steps
- Handle failures gracefully

This approach not only saves time but also ensures a systematic search across multiple sites - perfect for finding the best deals on shows and flights. More importantly, it shows how we can break down a process into smaller steps to increase our chances of success.

This pattern can be extended to other types of event planning or multi-step web tasks.

Happy planning! 🎭✈️